In [4]:
!pip install fitz frontend --quiet
!pip install --upgrade pymupdf --quiet
!pip install google-generativeai --quiet
!pip install langchain-core langchain langchain-community langchain-cohere --quiet
!pip install chromadb --quiet
!pip install tiktoken --quiet
!pip install pillow --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.27.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.28.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.67.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.5 which is incompatible.


In [5]:
import fitz  # PyMuPDF
from PIL import Image
import io
import os
from dotenv import load_dotenv

import google.generativeai as genai
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_cohere import ChatCohere, CohereEmbeddings

load_dotenv()

/home/marciacunha/Área de Trabalho/GUPY_DATA_SCIENCE/_sagemaker_ranking_candidates_formation/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

Download the "Attention is all you need" paper

In [6]:
!wget https://arxiv.org/pdf/1706.03762
!mv 1706.03762 attention_is_all_you_need.pdf

--2024-10-22 12:49:27--  https://arxiv.org/pdf/1706.03762
Resolvendo arxiv.org (arxiv.org)... 151.101.131.42, 151.101.3.42, 151.101.195.42, ...
Conectando-se a arxiv.org (arxiv.org)|151.101.131.42|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 2215244 (2,1M) [application/pdf]
Salvando em: ‘1706.03762’

1706.03762          100%[===================>]   2,11M  --.-KB/s    em 0,07s   

2024-10-22 12:49:28 (28,8 MB/s) - ‘1706.03762’ salvo [2215244/2215244]



Data Extraction

In [7]:
import fitz  # PyMuPDF
import os
from PIL import Image
import io
import glob

text_data = []
img_data = []
    

class PDFExtractor:
    
    def __init__(self, pdf_paths):
        """
        Inicializa a classe com uma lista de caminhos de arquivos PDF.
        :param pdf_paths: Lista de caminhos de arquivos PDF
        """
        self.pdf_paths = pdf_paths
        self.text_data = []

    def extract_from_pdfs(self):
        """
        Extrai texto e imagens de todos os PDFs da lista pdf_paths.
        """
        for pdf_path in self.pdf_paths:
            self.extract_from_single_pdf(pdf_path)

    def extract_from_single_pdf(self, pdf_path):
        """
        Extrai texto e imagens de um único PDF.
        :param pdf_path: Caminho do arquivo PDF
        """
        with fitz.open(pdf_path) as pdf_file:
            # Criar um diretório para armazenar as imagens, se não existir
            if not os.path.exists("extracted_images"):
                os.makedirs("extracted_images")

            # Iterar sobre cada página do PDF
            for page_number in range(len(pdf_file)):
                page = pdf_file[page_number]

                # Obter o texto da página
                text = page.get_text().strip()
                self.text_data.append({"response": text, "name": f"{os.path.basename(pdf_path)}_page_{page_number + 1}"})

                # Obter a lista de imagens na página
                images = page.get_images(full=True)

                # Iterar sobre todas as imagens encontradas na página
                for image_index, img in enumerate(images, start=0):
                    xref = img[0]  # Obter o XREF da imagem
                    base_image = pdf_file.extract_image(xref)  # Extrair a imagem
                    image_bytes = base_image["image"]  # Obter os bytes da imagem
                    image_ext = base_image["ext"]  # Obter a extensão da imagem

                    # Carregar a imagem usando PIL e salvá-la
                    image = Image.open(io.BytesIO(image_bytes))
                    image.save(f"extracted_images/{os.path.basename(pdf_path)}_page_{page_number + 1}_{image_index + 1}.{image_ext}")

    def get_text_data(self):
        """
        Retorna o texto extraído de todos os PDFs processados.
        """
        return self.text_data

# Caminho do diretório que contém os arquivos PDF
directory_path = 'PATH'

# Usando glob para listar todos os arquivos PDF no diretório
pdf_files = glob.glob(os.path.join(directory_path, "*.pdf"))

# Exemplo de uso:
extractor = PDFExtractor(pdf_files)
extractor.extract_from_pdfs()
text_data = extractor.get_text_data()

In [8]:
#https://github.com/google-gemini/cookbook?tab=readme-ov-file
#https://aistudio.google.com/app/apikey

import os
import google.generativeai as genai


class GenAIConfigurator:
    def __init__(self, api_key=None, model_name="gemini-1.5-flash"):
        """
        Inicializa a classe de configuração para genai com a chave da API e o modelo.

        :param api_key: Chave da API genai. Se não for fornecida, tentará buscar da variável de ambiente GENAI_API_KEY.
        :param model_name: Nome do modelo a ser usado. O padrão é "gemini-1.5-flash".
        """
        self.api_key = api_key or os.getenv('GENAI_API_KEY')
        self.model_name = model_name
        self.model = None

        if not self.api_key:
            raise ValueError("A chave da API deve ser fornecida ou estar definida na variável de ambiente 'GENAI_API_KEY'.")

    def configure(self):
        """
        Configura o cliente genai com a chave da API e o modelo especificado.
        """
        genai.configure(api_key=self.api_key)
        self.model = genai.GenerativeModel(model_name=self.model_name)
        print(f"Modelo {self.model_name} configurado com sucesso.")

    def get_model(self):
        """
        Retorna o modelo configurado.
        """
        if not self.model:
            raise RuntimeError("O modelo não foi configurado. Chame o método configure() primeiro.")
        return self.model

# Exemplo de uso:
configurator = GenAIConfigurator(api_key='YOUR_KEY')
configurator.configure()
model = configurator.get_model()

Modelo gemini-1.5-flash configurado com sucesso.


Image Captioning


In [9]:
import os
from PIL import Image

class ImageSummarizer:
    def __init__(self, image_dir, model):
        """
        Inicializa a classe para processar as imagens e gerar resumos.

        :param image_dir: Diretório onde as imagens estão armazenadas.
        :param model: Modelo generativo configurado para gerar os resumos.
        """
        self.image_dir = image_dir
        self.model = model
        self.img_data = []

        if not os.path.exists(image_dir):
            raise ValueError(f"O diretório {image_dir} não existe.")

    def summarize_images(self):
        """
        Itera sobre as imagens no diretório e gera resumos usando o modelo.
        """
        for img_file in os.listdir(self.image_dir):
            img_path = os.path.join(self.image_dir, img_file)
            image = Image.open(img_path)
            response = self.model.generate_content([
                image,
                "You are an assistant tasked with summarizing tables, images and text for retrieval. These summaries will be embedded and used to retrieve the raw text or table elements. Give a concise summary of the table or text that is well optimized for retrieval. Table or text or image:"
            ])
            self.img_data.append({"response": response.text, "name": img_file})

    def get_summaries(self):
        """
        Retorna os resumos gerados para todas as imagens.

        :return: Lista de dicionários contendo os resumos e os nomes das imagens.
        """
        return self.img_data

# Exemplo de uso:
# model = <instância do modelo configurado anteriormente>
image_summarizer = ImageSummarizer(image_dir="extracted_images", model=model)
image_summarizer.summarize_images()
summaries = image_summarizer.get_summaries()


In [10]:
summaries

[{'response': 'This image shows a scaled dot-product attention mechanism, which is used to compute the attention weights that are then applied to a value matrix. The mechanism first calculates the dot product of a query matrix and a key matrix. The result is then scaled and masked. Finally, a softmax operation is applied to compute the attention weights. The attention weights are then multiplied by the value matrix. The inputs to the scaled dot-product attention are Q, K, and V, which stand for query, key, and value.',
  'name': 'image_4_1.png'},
 {'response': 'This image illustrates the architecture of a transformer model, commonly used in natural language processing.  The input to the model is a sequence of words, which are first embedded into a vector space and then augmented with positional encodings. These encodings are processed in a series of encoder layers, each of which consists of two sub-layers: a multi-head attention layer and a feed-forward neural network. The encoder laye

Vectostore


In [11]:
# Set embeddings
#https://python.langchain.com/docs/integrations/text_embedding/cohere/

import os
import getpass
from langchain_cohere import ChatCohere, CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

class DocumentImageProcessor:
    def __init__(self, api_key=None, embedding_model_name="embed-english-v3.0", collection_name="multi_model_rag"):
        """
        Inicializa a classe para configurar a chave da API Cohere e processar documentos e imagens.

        :param api_key: Chave da API Cohere. Se não for fornecida, tenta buscar da variável de ambiente COHERE_API_KEY.
        :param embedding_model_name: Nome do modelo de embeddings da Cohere. Padrão: "embed-english-v3.0".
        """
        self.api_key = api_key or os.getenv("COHERE_API_KEY")
        self.embedding_model_name = embedding_model_name
        self.embedding_model = None
        self.collection_name = collection_name
        self.vectorstore = None

        if not self.api_key:
            raise ValueError("A chave da API deve ser fornecida ou definida na variável de ambiente 'COHERE_API_KEY'.")
        self._set_api_key()

    def _set_api_key(self):
        """
        Define a chave da API no ambiente se não estiver presente.
        """
        if not os.getenv("COHERE_API_KEY"):
            os.environ["COHERE_API_KEY"] = self.api_key

    def configure_model(self):
        """
        Configura o modelo de embeddings da Cohere.
        """
        self.embedding_model = CohereEmbeddings(model=self.embedding_model_name)
        print(f"Modelo de embeddings {self.embedding_model_name} configurado com sucesso.")

    def process_documents_and_images(self, text_data, img_data, chunk_size=400, chunk_overlap=50):
        """
        Carrega e divide documentos e imagens em partes menores.

        :param text_data: Lista de textos para criar os documentos.
        :param img_data: Lista de resumos de imagens para criar os documentos.
        :param chunk_size: Tamanho dos chunks após a divisão.
        :param chunk_overlap: Quantidade de sobreposição entre os chunks.
        :return: Documentos e imagens divididos em partes menores.
        """
        # Carregar documentos e imagens
        docs_list = [Document(page_content=text['response'], metadata={"name": text['name']}) for text in text_data]
        img_list = [Document(page_content=img['response'], metadata={"name": img['name']}) for img in img_data]

        # Dividir em chunks menores
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap
        )

        doc_splits = text_splitter.split_documents(docs_list)
        img_splits = text_splitter.split_documents(img_list)

        return doc_splits, img_splits
    
    def add_to_vectorstore(self, doc_splits, img_splits):
        """
        Adiciona os documentos ao vectorstore usando Chroma.

        :param doc_splits: Divisões de documentos de texto.
        :param img_splits: Divisões de documentos de imagem.
        """
        # Adicionando ao vectorstore
        self.vectorstore = Chroma.from_documents(
            documents=doc_splits + img_splits,  # texto e imagens
            collection_name=self.collection_name,
            embedding=self.embedding_model
        )

    def get_retriever(self, k=1):
        """
        Cria um retriever para buscar documentos similares.

        :param k: Número de documentos a recuperar (padrão 1).
        :return: Um retriever configurado.
        """
        if self.vectorstore is None:
            raise ValueError("Vectorstore não foi inicializado. Adicione os documentos primeiro.")

        # Criando o retriever com base na similaridade
        retriever = self.vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={'k': k}
        )
        return retriever
    

# Exemplo de uso:
# text_data e img_data devem ser listas de dicionários com o conteúdo de resposta e nome.
processor = DocumentImageProcessor(api_key='YOUR_KEY')
processor.configure_model()
doc_splits, img_splits = processor.process_documents_and_images(text_data, img_data)
# Adicionando ao vectorstore
processor.add_to_vectorstore(doc_splits, img_splits)

# Criando retriever
retriever = processor.get_retriever(k=1)


Modelo de embeddings embed-english-v3.0 configurado com sucesso.


In [12]:
retriever

VectorStoreRetriever(tags=['Chroma', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7587a155ee20>, search_kwargs={'k': 1})

In [16]:
img_splits

[]

Query |Output


In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere, CohereEmbeddings

load_dotenv()

class RAGQuery:
    def __init__(self, retriever, llm_model="command-r-plus", temperature=0, system_message=None):
        """
        Inicializa a classe com o retriever e o LLM.

        :param retriever: O retriever configurado para buscar documentos.
        :param llm_model: Nome do modelo LLM a ser usado (padrão: "command-r-plus").
        :param temperature: Valor de temperatura para controle de criatividade do LLM (padrão 0).
        :param system_message: Mensagem padrão do sistema a ser usada no prompt (padrão é um sistema para tarefas de QA).
        """
        self.retriever = retriever
        self.llm = ChatCohere(model=llm_model, temperature=temperature)

        # Definindo a mensagem do sistema
        self.system_message = system_message or """You are an assistant for question-answering tasks. Answer the question based upon your knowledge. 
        Use three-to-five sentences maximum and keep the answer concise."""

    def run_query(self, query, max_docs=1):
        """
        Realiza uma consulta ao retriever, cria o prompt e executa a geração da resposta.

        :param query: A pergunta feita pelo usuário.
        :param max_docs: Número máximo de documentos a recuperar (padrão 1).
        :return: A resposta gerada pelo LLM.
        """
        # Recuperando os documentos
        docs = self.retriever.invoke(query)[:max_docs]

        # Definindo o template de prompt
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.system_message),
                ("human", "Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question>"),
            ]
        )

        # Criando o encadeamento RAG
        rag_chain = prompt | self.llm | StrOutputParser()

        # Executando a geração
        generation = rag_chain.invoke({"documents": docs[0].page_content, "question": query})
        return generation

# Exemplo de uso:
if __name__ == "__main__":
    # Suponha que 'retriever' já esteja configurado
    query = "What is the BLEU score of the Transformer (base model)?"
    
    # Criando instância da classe
    rag_query_class = RAGQuery(retriever)

    # Executando a consulta
    answer = rag_query_class.run_query(query)
    print(answer)


The Transformer (base model) achieves a BLEU score of 27.3.
